In [1]:
import pandas as pd
from pathlib import Path
import re
import numpy as np


## Merging movie reviews into one dataframe

In [2]:
data_dir = Path("dataset/2_reviews_per_movie_raw")

dfs = [
    pd.read_csv(file).assign(movie=file.stem)
    for file in data_dir.glob("*.csv")
]

df_reviews = pd.concat(dfs, ignore_index=True)


## Creating a mapping from imdb movie ids to movies in movie reviews dataset

In [3]:
def extract_year(title):
    match = re.search(r'\((\d{4})\)$', title)
    return match.group(1) if match else None

In [4]:

def extract_clean_title(title):
    return re.sub(r'\s*\(\d{4}\)$', '', title).strip()

In [5]:
ratings_path = Path("ml-32m/movies.csv")
df_ratings = pd.read_csv(ratings_path)

df_ratings['year'] = df_ratings['title'].apply(extract_year)
df_ratings['clean_title'] = df_ratings['title'].apply(extract_clean_title)

movie_mapping = {
    (row['clean_title'], row['year']): row['movieId']
    for _, row in df_ratings.iterrows()
}


## Set movie ids to reviews, drop reviews for movies not present in imdb dataset

In [6]:
def split_movie_and_movie_year(movie_str):
    match = re.search(r'(\d{4})$', movie_str)
    if match:
        year = match.group(1)
        title = movie_str[:match.start()].strip()
        return title, year
    return movie_str, None

In [7]:

df_reviews[['clean_movie', 'movie_year']] = df_reviews['movie'].apply(
    lambda x: pd.Series(split_movie_and_movie_year(x))
)

df_reviews['movieId'] = df_reviews.apply(
    lambda row: movie_mapping.get((row['clean_movie'], row['movie_year'])), axis=1
)

df_reviews.dropna(inplace=True)

In [8]:
df_reviews["review"]  = df_reviews.apply(lambda row: "".join([row["title"], row["review"]]), axis=1)

In [9]:
df_reviews

,username,rating,helpful,total,date,title,review,movie,clean_movie,movie_year,movieId
737,boognish-6,10,252,291,4 April 2006,The Most Underrated Film of the '90s\n,The Most Underrated Film of the '90s\nThe rev...,The Devil's Advocate 1997,The Devil's Advocate,1997,1645.0
738,EdwardsMovieKit,9,155,180,2 July 2006,All Quality Elements of a Magnificent Movie!!!\n,All Quality Elements of a Magnificent Movie!!...,The Devil's Advocate 1997,The Devil's Advocate,1997,1645.0
739,Special-K88,Null,175,208,12 April 2002,this is how you make a thriller\n,"this is how you make a thriller\nStylish, sex...",The Devil's Advocate 1997,The Devil's Advocate,1997,1645.0
740,ereinion,9,89,105,1 April 2005,A beguiling experience\n,A beguiling experience\nThis is one of the mo...,The Devil's Advocate 1997,The Devil's Advocate,1997,1645.0
741,bsamdahl,9,40,46,21 February 2002,"Vanity, my favorite sin\n","Vanity, my favorite sin\nThe Devil's advocate...",The Devil's Advocate 1997,The Devil's Advocate,1997,1645.0
...,...,...,...,...,...,...,...,...,...,...,...
928726,Mackiatoe,6,0,12,3 April 2015,"The ending the series deserved, even if it wa...","The ending the series deserved, even if it wa...",Furious 7 2015,Furious 7,2015,130634.0
928727,olivergilbert1,8,0,2,3 April 2015,For Paul\n,For Paul\nThe latest instalment of the fast &...,Furious 7 2015,Furious 7,2015,130634.0
928728,Myjoyandlight,7,0,12,3 April 2015,Bittersweet movie with over-the-top action!\n,Bittersweet movie with over-the-top action!\n...,Furious 7 2015,Furious 7,2015,130634.0
928729,BrentHankins,7,0,13,2 April 2015,"Plenty of the franchise's trademark action, p...","Plenty of the franchise's trademark action, p...",Furious 7 2015,Furious 7,2015,130634.0


In [10]:
df_reviews["rating"] = df_reviews["rating"].apply(lambda x: int(x) if x != "Null" else np.nan)

In [11]:
df_reviews.dropna(inplace=True)

In [12]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 586018 entries, 737 to 928730
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   username     586018 non-null  object 
 1   rating       586018 non-null  float64
 2   helpful      586018 non-null  int64  
 3   total        586018 non-null  int64  
 4   date         586018 non-null  object 
 5   title        586018 non-null  object 
 6   review       586018 non-null  object 
 7   movie        586018 non-null  object 
 8   clean_movie  586018 non-null  object 
 9   movie_year   586018 non-null  object 
 10  movieId      586018 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 53.7+ MB


## Processing ratings

In [13]:
reviews_by_movie = {
    movie: group.reset_index(drop=True)
    for movie, group in df_reviews.groupby('movieId')
}

In [14]:
df_reviews

,username,rating,helpful,total,date,title,review,movie,clean_movie,movie_year,movieId
737,boognish-6,10.0,252,291,4 April 2006,The Most Underrated Film of the '90s\n,The Most Underrated Film of the '90s\nThe rev...,The Devil's Advocate 1997,The Devil's Advocate,1997,1645.0
738,EdwardsMovieKit,9.0,155,180,2 July 2006,All Quality Elements of a Magnificent Movie!!!\n,All Quality Elements of a Magnificent Movie!!...,The Devil's Advocate 1997,The Devil's Advocate,1997,1645.0
740,ereinion,9.0,89,105,1 April 2005,A beguiling experience\n,A beguiling experience\nThis is one of the mo...,The Devil's Advocate 1997,The Devil's Advocate,1997,1645.0
741,bsamdahl,9.0,40,46,21 February 2002,"Vanity, my favorite sin\n","Vanity, my favorite sin\nThe Devil's advocate...",The Devil's Advocate 1997,The Devil's Advocate,1997,1645.0
742,Snoopymichele,10.0,17,18,21 July 2006,Be careful what you wish for\n,Be careful what you wish for\nWhen Kevin Loma...,The Devil's Advocate 1997,The Devil's Advocate,1997,1645.0
...,...,...,...,...,...,...,...,...,...,...,...
928726,Mackiatoe,6.0,0,12,3 April 2015,"The ending the series deserved, even if it wa...","The ending the series deserved, even if it wa...",Furious 7 2015,Furious 7,2015,130634.0
928727,olivergilbert1,8.0,0,2,3 April 2015,For Paul\n,For Paul\nThe latest instalment of the fast &...,Furious 7 2015,Furious 7,2015,130634.0
928728,Myjoyandlight,7.0,0,12,3 April 2015,Bittersweet movie with over-the-top action!\n,Bittersweet movie with over-the-top action!\n...,Furious 7 2015,Furious 7,2015,130634.0
928729,BrentHankins,7.0,0,13,2 April 2015,"Plenty of the franchise's trademark action, p...","Plenty of the franchise's trademark action, p...",Furious 7 2015,Furious 7,2015,130634.0


In [15]:
df_reviews = df_reviews[df_reviews['rating'] != "Null"].copy()
df_reviews['rating'] = pd.to_numeric(df_reviews['rating'], errors='coerce')
df_reviews = df_reviews.dropna(subset=['rating'])
df_reviews['scaled_rating'] = 0.5 + 0.5 * (df_reviews['rating'] - 1)

reviews_by_movie = {
    movie: group.reset_index(drop=True)
    for movie, group in df_reviews.groupby('movieId')
}

## Assign available reviews

We assign reviews to entries in our dataset of ratings in such a way that reviews are not repeated and the difference between the rating in dataset and rating in review doesnt differ in more than 2 stars out of 5 (so that there is no extreme cases such that 1 star rating has 5 star review)

In [ ]:
def assign_reviews_unique_diff(ratings_group, reviews_group, lower_bound=0, upper_bound=1.5):
    ratings_group = ratings_group.copy()
    n_ratings = len(ratings_group)
    
    assigned_usernames = [None] * n_ratings
    assigned_reviews   = [None] * n_ratings
    assigned_raw_ratings = [None] * n_ratings
    assigned_diffs     = [None] * n_ratings

    if reviews_group is None or reviews_group.empty:
        ratings_group['review_username'] = assigned_usernames
        ratings_group['review_text'] = assigned_reviews
        ratings_group['review_raw_rating'] = assigned_raw_ratings
        ratings_group['rating_diff'] = assigned_diffs
        return ratings_group

    available = list(range(len(reviews_group)))
    
    review_scaled = reviews_group['scaled_rating'].values
    review_usernames = reviews_group['username'].values
    review_texts = reviews_group['review'].values
    review_raw_ratings = reviews_group['rating'].values

    for i, imdb_rating in enumerate(ratings_group['rating']):
        best_idx = None
        best_diff = None
        for j in available:
            diff = abs(imdb_rating - review_scaled[j])
            if lower_bound <= diff <= upper_bound:
                if best_idx is None or diff < best_diff:
                    best_idx = j
                    best_diff = diff
        if best_idx is not None:
            assigned_usernames[i] = review_usernames[best_idx]
            assigned_reviews[i] = review_texts[best_idx]
            assigned_raw_ratings[i] = review_raw_ratings[best_idx]
            assigned_diffs[i] = best_diff
            available.remove(best_idx)
            if not available:
                break

    ratings_group['review_username'] = assigned_usernames
    ratings_group['review_text'] = assigned_reviews
    ratings_group['review_raw_rating'] = assigned_raw_ratings
    ratings_group['rating_diff'] = assigned_diffs
    return ratings_group



In [20]:
df_ratings = pd.read_csv("ml-32m/ratings.csv")

df_ratings_with_diff_reviews = df_ratings.groupby('movieId', group_keys=False).apply(
    lambda group: assign_reviews_unique_diff(group, reviews_by_movie.get(group.name), lower_bound=0, upper_bound=2)
)

/var/folders/7f/gc2d7y611gn1hg8cn9hd196h0000gp/T/ipykernel_41107/1230877411.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_ratings_with_diff_reviews = df_ratings.groupby('movieId', group_keys=False).apply(
/var/folders/7f/gc2d7y611gn1hg8cn9hd196h0000gp/T/ipykernel_41107/1230877411.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ratings_with_diff_reviews = df_ratings.groupby('movieId', group_keys=False).apply(


In [21]:
df_ratings_with_diff_reviews

,userId,movieId,rating,timestamp,review_username,review_text,review_raw_rating,rating_diff
0,1,17,4.0,944249077,None,None,NaN,NaN
1,1,25,1.0,944250228,None,None,NaN,NaN
2,1,29,2.0,943230976,None,None,NaN,NaN
3,1,30,5.0,944249077,None,None,NaN,NaN
4,1,32,5.0,943228858,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...
32000199,200948,79702,4.5,1294412589,None,None,NaN,NaN
32000200,200948,79796,1.0,1287216292,None,None,NaN,NaN
32000201,200948,80350,0.5,1294412671,None,None,NaN,NaN
32000202,200948,80463,3.5,1350423800,None,None,NaN,NaN


In [22]:
df_ratings_with_diff_reviews[df_ratings_with_diff_reviews["rating_diff"] > 0]

,userId,movieId,rating,timestamp,review_username,review_text,review_raw_rating,rating_diff
70,1,1242,1.0,943231162,harry-76,Distorted Perspective\nMilitary intelligence ...,3.0,0.5
8839,61,595,3.0,831973763,Atreyu_II,A very popular Disney movie\nThe 30th animate...,7.0,0.5
10460,70,595,3.0,996496466,jessbir-23618,be our guest\nBeauty and the beast is a timel...,7.0,0.5
11172,73,1293,1.0,1172040582,ldlAkasha,historically inaccurate\nThis movie had poten...,1.0,0.5
11973,77,595,3.0,842069194,spencergrande6,"Plot is flimsy but the music is iconic, duh\n...",7.0,0.5
...,...,...,...,...,...,...,...,...
31966675,200724,183869,2.5,1604015287,covington_c,Really though!?\nUnbelievable craptacular bs....,1.0,2.0
31967251,200726,143347,4.0,1604374451,bladsgaard,One of the very best superhero movies so far!...,9.0,0.5
31968207,200729,200540,3.5,1572128608,rima1981,"Awesome!\nIt was funny, it was sweet and all ...",9.0,1.0
31987956,200861,197711,2.5,1576772827,neptunesblood,Worst Horror Movie in a long time.\nI conside...,1.0,2.0


In [23]:
df_ratings_with_diff_reviews.drop(columns=["timestamp", "review_username"], inplace=True)

In [24]:
df_ratings_with_diff_reviews

,userId,movieId,rating,review_text,review_raw_rating,rating_diff
0,1,17,4.0,None,NaN,NaN
1,1,25,1.0,None,NaN,NaN
2,1,29,2.0,None,NaN,NaN
3,1,30,5.0,None,NaN,NaN
4,1,32,5.0,None,NaN,NaN
...,...,...,...,...,...,...
32000199,200948,79702,4.5,None,NaN,NaN
32000200,200948,79796,1.0,None,NaN,NaN
32000201,200948,80350,0.5,None,NaN,NaN
32000202,200948,80463,3.5,None,NaN,NaN


In [25]:
df_ratings_with_diff_reviews[df_ratings_with_diff_reviews["review_text"].notna()]

,userId,movieId,rating,review_text,review_raw_rating,rating_diff
8,1,110,3.0,Mel versus history\nWhy is Braveheart regarde...,6.0,0.0
9,1,111,5.0,"Disturbing, powerful, relevant, important\nA ...",10.0,0.0
15,1,260,5.0,"A classics, phenomenal and arguably the best ...",10.0,0.0
21,1,356,2.0,Assault on intellectuals everywhere.\nPeople ...,4.0,0.0
22,1,527,3.0,Overrated!\nIt's not that Schindler's List is...,6.0,0.0
...,...,...,...,...,...,...
31987956,200861,197711,2.5,Worst Horror Movie in a long time.\nI conside...,1.0,2.0
31987959,200861,200540,1.5,Bad.\n<ul><li>Confusing. If you hadn't seen t...,1.0,1.0
31987968,200861,203222,0.5,Sticking to the original movie (1994)\nThe 20...,1.0,0.0
31990207,200876,194951,0.5,Direction and script broken. Zero understandi...,1.0,0.0


In [37]:
df_ratings_with_diff_reviews.to_csv("data/ratings_reviews_unique.csv", index=False)

In the end we have dataset with user id, movie id, rating and review text where applicable. Overall we have 573k reviews and 32m rows with ratings. There are also debug columns for review rating from the other source with reviews and scaled rating difference between rating and review rating. Next step is to obtain semantic score for available reviews and use them in our recommendation system